# Analyze multiple video and compare results
Nikola Bátová & Jesse ten Broeke

n.batova@student.rug.nl

j.ten.broeke1@student.rug.nl

Necessary imports

In [11]:
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')
from functions_for_DLC_analysis import get_bodyparts, create_rois, get_total_exploration_time, get_exploration_time

### Define DLCscorer and load video data
Change the DLCscorer variable according to your dataframe. You can find this by entering your 'videos' folder and reading it off the repeating sequence in the analysed data from your videos.

Before running this cell, make sure you have created this video_data.csv file by running set_ratios.ipynb.

In [12]:
# Variable for the Dataframe to locate .h5 data
DLCscorer = 'DLC_resnet50_BN research projectFeb3shuffle1_54500'

video_data = pd.read_csv(f'video_data.csv')

video_names = video_data['videoname']
cm_to_pixel_ratios = (float(i) for i in video_data['cm_to_pixel_ratio'])

In [13]:
# Bugfix to take care of numpy nep50 warning.
def dummy_npwarn_decorator_factory():
  def npwarn_decorator(x):
    return x
  return npwarn_decorator
np._no_nep50_warning = getattr(np, '_no_nep50_warning', dummy_npwarn_decorator_factory)

### Defining counting and not counting body parts, and frames per second
Edit the fps variable depending on how many frames per second your analysed videos have. Default is 30.

Edit the bpt_plus_names list with the body parts you want whose times in regions of interest you want to count in to the exploration time.

Edit the bpt_mins_names list with the body parts you want whose times you want to substract from the exploration time.

In [14]:
# Frames per seconds in the videos being scored (default 30)
fps=30

# ADD body parts whose time you want to track in your ROI
bpt_plus_names=['nose']

# ADD body parts whose time you want to substract from the total time
bpt_minus_names=['left-front-paw', 'right-front-paw']

### Adding Regions of Interest (ROIs)
Add the objects in regions of interest how you have them named in your labeling and the distance from your object, which still counts as your ROI


In [15]:
# ADD a list of names of ROIs
rois_names = ['non-moved object', 'moved object']

# ADD how many cm from your object you want to track you body part, default 1
boundary_size_distance = 1

### Calculate and Print Results

In [ ]:
results = []
for video, ratio in zip(video_names, cm_to_pixel_ratios):
    print(f'Video {video}:')

    df= pd.read_hdf(f'../videos/{video}DLC_resnet50_BN research projectFeb3shuffle1_54500.h5')

    rois = create_rois(df=df, DLCscorer=DLCscorer, distance=boundary_size_distance, names=rois_names, cm_to_pixel_ratio=ratio)

    # Create rois for which time will be substracted if a certain body part is in one of these ROIs
    rois_substracting = create_rois(df=df, DLCscorer=DLCscorer, distance=0, names=rois_names, cm_to_pixel_ratio=ratio)

    # Ectract time, velocity & x and y coordinates for each body part
    bpt_plus = get_bodyparts(df=df, DLCscorer=DLCscorer, fps=fps, bpt_names=bpt_plus_names)

    # Ectract time, velocity & x and y coordinates for each body part
    bpt_minus = get_bodyparts(df=df, DLCscorer=DLCscorer, fps=fps, bpt_names=bpt_minus_names)


    explorationn = get_exploration_time(rois, rois_substracting, bpt_plus, bpt_minus)
    for exp in explorationn:
        print(f"Total Exploration Time for {exp.get('name')}: {exp.get('exploration time')}s")
        results.append({
            "video_name": video,
            "roi_name": exp.get("name"),
            "exploration_time": exp.get("exploration time"),
            "cm_to_pixel_ratio": ratio
        })
    print("\n")
    

Video 233 (1):
Total Exploration Time for non-moved object: 8.933333333333328s
Total Exploration Time for moved object: 7.433333333333333s


Video 234 (1):
Total Exploration Time for non-moved object: 0.06666666666667709s
Total Exploration Time for moved object: 0s


Video 235 (1):
Total Exploration Time for non-moved object: 14.433333333333335s
Total Exploration Time for moved object: 20.4s


Video 236 (1):
Total Exploration Time for non-moved object: 0s
Total Exploration Time for moved object: 9.699999999999998s


Video 237 (1):
Total Exploration Time for non-moved object: 2.966666666666666s
Total Exploration Time for moved object: 21.133333333333333s


Video 238 (1):
Total Exploration Time for non-moved object: 29.433333333333334s
Total Exploration Time for moved object: 4.500000000000002s


Video 239 (1):
Total Exploration Time for non-moved object: 0s
Total Exploration Time for moved object: 3.3333333333333335s




### Save results in a .csv file

In [17]:
# Convert results to a DataFrame
df_results = pd.DataFrame(results)

# Save to CSV
df_results.to_csv("exploration_times.csv", index=False)